In [1]:
pip install git+https://github.com/huggingface/setfit.git

  Cloning https://github.com/huggingface/setfit.git to /private/var/folders/ld/mczz0xmx243grk32lg665c_c0000gn/T/pip-req-build-gvnt3953
  Running command git clone --quiet https://github.com/huggingface/setfit.git /private/var/folders/ld/mczz0xmx243grk32lg665c_c0000gn/T/pip-req-build-gvnt3953
  Resolved https://github.com/huggingface/setfit.git to commit 327a3b64ca38dd688a7f92b67e7bf88f9db66aa9
  Preparing metadata (setup.py) ... done
  Created wheel for setfit: filename=setfit-1.1.0.dev0-py3-none-any.whl size=75556 sha256=30baf444ad1ff6fe0648c78bc84dd72bc45ce26731aa72e2b14096978bbeac6a
  Stored in directory: /private/var/folders/ld/mczz0xmx243grk32lg665c_c0000gn/T/pip-ephem-wheel-cache-wxoa9vyw/wheels/e6/90/ff/ac72ac5cb7aea7e840f8056d4464d8a1ddc41a54369604c58b
Successfully built setfit
  Attempting uninstall: setfit
    Found existing installation: setfit 1.0.3
    Uninstalling setfit-1.0.3:
      Successfully uninstalled setfit-1.0.3
Note: you may need to restart the kernel to use upd

In [18]:
import pandas as pd
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [3]:
#Load dataset
df = pd.read_csv('preprocessed_data.csv')

In [6]:
# Check dataset
df.head()

,ID,entity,ent_id,sentence_original,class_ID,class_name,sentence_id,sentence_preprocessed,sentence_stemmed,sentence_lemmas,sentence_tokenized
0,90812,interviews,qualitative interview,Especially for the problem of reduced identity...,1,belongs_to_article,507_6563_6594,especially problem reduced identity efforts vt...,especi problem reduc ident effort vt identifi ...,especially problem reduce identity effort vt i...,"['especially', 'problem', 'reduced', 'identity..."
1,90770,knowledge management,knowledge management,This makes it challenging for them to organize...,3,background_information,507_4787_4813,makes challenging organize meetings conversati...,make challeng organ meet convers well perform ...,make challenge organize meeting conversation w...,"['makes', 'challenging', 'organize', 'meetings..."
2,90500,reference model,reference modelling,It can be foreseen that managing such a compre...,3,background_information,505_7541_7567,foreseen managing comprehensive reference mode...,foreseen manag comprehens refer model challeng...,foresee manage comprehensive reference model c...,"['fore', '##see', '##n', 'managing', 'comprehe..."
3,90417,UML,unified modeling language,"In particular, redefinition in UML allows one ...",3,background_information,505_4085_4121,particular redefinition uml allows one modify ...,particular redefinit uml allow on modifi data ...,particular redefinition uml allow one modify d...,"['particular', 'red', '##ef', '##ini', '##tion..."
4,90361,reference modeling,reference modelling,"At the same time, redundancy in a reference mo...",2,related_work,505_2464_2508,time redundancy reference model typically avoi...,time redund refer model typic avoid see e g co...,time redundancy reference model typically avoi...,"['time', 'red', '##unda', '##ncy', 'reference'..."


In [9]:
# Extract needed columns
df = df[['entity', 'sentence_original', 'class_ID']]
df.head()

,entity,sentence_original,class_ID
0,interviews,Especially for the problem of reduced identity...,1
1,knowledge management,This makes it challenging for them to organize...,3
2,reference model,It can be foreseen that managing such a compre...,3
3,UML,"In particular, redefinition in UML allows one ...",3
4,reference modeling,"At the same time, redundancy in a reference mo...",2


In [22]:
# Combine entity and sentence into one text column
df['text'] = df.apply(lambda row: f"Entity: {row['entity']}, Sentence: {row['sentence_original']}", axis=1)

# Rename 'class_ID' to 'label' to match SetFit expectations
df = df.rename(columns={'class_ID': 'label'})

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the DataFrame to a Dataset object
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

In [38]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 800
})

In [24]:
from setfit import SetFitModel, Trainer, sample_dataset
# Initialize the SetFit model
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [30]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer



# Create a Trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss
)

# Train the model
trainer.train()

/var/folders/ld/mczz0xmx243grk32lg665c_c0000gn/T/ipykernel_39725/1935454956.py:7: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
  6%|▋         | 126/2000 [05:01<1:14:50,  2.40s/it]
***** Running training *****
  Num unique pairs = 32000
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 2000
  0%|          | 0/2000 [00:00<?, ?it/s]

                                               

{'embedding_loss': 0.1735, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.0}


{'embedding_loss': 0.1897, 'learning_rate': 5e-06, 'epoch': 0.03}


{'embedding_loss': 0.0951, 'learning_rate': 1e-05, 'epoch': 0.05}


{'embedding_loss': 0.1194, 'learning_rate': 1.5000000000000002e-05, 'epoch': 0.07}


...

{'embedding_loss': 0.0, 'learning_rate': 1.111111111111111e-06, 'epoch': 0.95}


{'embedding_loss': 0.0, 'learning_rate': 5.555555555555555e-07, 'epoch': 0.97}


{'embedding_loss': 0.0001, 'learning_rate': 0.0, 'epoch': 1.0}



100%|██████████| 2000/2000 [22:51<00:00,  1.46it/s]


{'train_runtime': 1372.0012, 'train_samples_per_second': 23.324, 'train_steps_per_second': 1.458, 'epoch': 1.0}


In [31]:
# Evaluate the model on the test set
results = trainer.evaluate()
print(f"Evaluation results: {results}")

***** Running evaluation *****

Evaluation results: {'accuracy': 0.9}


In [37]:
from sklearn.metrics import classification_report
from datasets import load_metric

# Load the metric
metric = load_metric("f1")

# Get predictions from the model
preds = model(test_dataset["text"])
labels = test_dataset["label"]

# Calculate F1 score
f1_score = metric.compute(predictions=preds, references=labels, average="weighted")
print(f"F1 score: {f1_score['f1']}")

# Define class names manually
class_names = ["1", "2", "3"]  # Replace with your actual class names

# Generate classification report
report = classification_report(labels, preds, target_names=class_names)
print(report)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


F1 score: 0.9004890679424231
              precision    recall  f1-score   support

           1       0.89      0.90      0.90        84
           2       0.98      0.89      0.93        47
           3       0.86      0.90      0.88        69

    accuracy                           0.90       200
   macro avg       0.91      0.90      0.90       200
weighted avg       0.90      0.90      0.90       200

